In [1]:
import os
import numpy as np
import mdtraj as md
import pickle
from simtk.openmm import unit
from tqdm import tqdm_notebook
from simtk.openmm import app
import matplotlib.pyplot as plt

In [2]:
def new_positions(htf, hybrid_positions):
    n_atoms_new = htf._topology_proposal.n_atoms_new
    hybrid_indices = [htf._new_to_hybrid_map[idx] for idx in range(n_atoms_new)]
    return hybrid_positions[hybrid_indices, :]
    
def old_positions(htf, hybrid_positions):
    n_atoms_old = htf._topology_proposal.n_atoms_old
    hybrid_indices = [htf._old_to_hybrid_map[idx] for idx in range(n_atoms_old)]
    return hybrid_positions[hybrid_indices, :]

In [3]:
def make_traj(outdir, htf, i, phase, replica=0):
    new_top = md.Topology.from_openmm(htf._topology_proposal.new_topology)
    old_top = md.Topology.from_openmm(htf._topology_proposal.old_topology)
    
    # From Hannah: https://github.com/hannahbrucemacdonald/endstate_pdbs/blob/master/scripts/input_for_pol_calc.py
    from perses.analysis.utils import open_netcdf
    nc = open_netcdf(os.path.join(outdir, f"{i}_{phase}.nc"))
    nc_checkpoint = open_netcdf(os.path.join(outdir, f"{i}_{phase}_checkpoint.nc"))
    checkpoint_interval = nc_checkpoint.CheckpointInterval
    all_positions = nc_checkpoint.variables['positions']
    n_iter, n_replicas, n_atoms, _ = np.shape(all_positions)
    box_vectors = np.array(nc_checkpoint['box_vectors'])
    
    from tqdm import tqdm_notebook
    all_pos_new = np.zeros(shape=(n_iter, new_top.n_atoms, 3))
    all_pos_old = np.zeros(shape=(n_iter, old_top.n_atoms, 3))
    all_pos_hybrid = np.zeros(shape=(n_iter, n_atoms, 3))
    n_iter = 840
    for iteration in tqdm_notebook(range(n_iter)):
        replica_id = np.where(nc.variables['states'][iteration*checkpoint_interval] == replica)[0]
#         replica_id = 0
        print(replica_id)
        pos = all_positions[iteration,replica_id,:,:][0] *unit.nanometers
        all_pos_new[iteration] = new_positions(htf, pos).value_in_unit_system(unit.md_unit_system) # Get new positions only
#         all_pos_hybrid[iteration] = pos # Get hybrid positions
#         all_pos_old[iteration] = old_positions(htf, pos).value_in_unit_system(unit.md_unit_system)

    
    return all_pos_new
       

## 31/30/0 (with 35 replicas)

In [5]:
outdir = "/data/chodera/zhangi/perses_benchmark/repex/31/30/0/35_replicas/"
i = 0
phase = 'complex'
name = 'asn'

with open(os.path.join(outdir, f"{i}_{phase}.pickle"), "rb") as f:
    htf = pickle.load(f)
    


In [24]:
all_pos_new = make_traj(outdir, htf, i, phase)

<ipython-input-23-39e34a7f0bfa>:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for iteration in tqdm_notebook(range(n_iter)):


  0%|          | 0/948 [00:00<?, ?it/s]

In [25]:
new_top = md.Topology.from_openmm(htf._topology_proposal.new_topology)

traj = md.Trajectory(np.array(all_pos_new), new_top)

box_vectors = np.array([val.value_in_unit_system(unit.md_unit_system) for val in htf.hybrid_system.getDefaultPeriodicBoxVectors()])

vectors = np.array([box_vectors]) 

# Set unit cell vectors in traj 
traj.unitcell_vectors = np.repeat(vectors, 948, axis=0)
traj.image_molecules(inplace=True)

traj.save(os.path.join(outdir, f"{i}_{phase}_new_replica_17.dcd"))
traj[0].save(os.path.join(outdir, f"{i}_{phase}_new_replica_17.pdb"))

ValueError: Buffer dtype mismatch, expected 'float' but got 'double'

Exception ignored in: 'mdtraj.geometry._geometry.image_frame'
Traceback (most recent call last):
  File "/home/zhangi/miniconda3/envs/perses-rbd-ace2/lib/python3.8/site-packages/mdtraj/core/trajectory.py", line 2003, in image_molecules
    _geometry.image_molecules(result.xyz, box, anchor_molecules_atom_indices, other_molecules_atom_indices, sorted_bonds)
ValueError: Buffer dtype mismatch, expected 'float' but got 'double'


ValueError: Buffer dtype mismatch, expected 'float' but got 'double'

Exception ignored in: 'mdtraj.geometry._geometry.image_frame'
Traceback (most recent call last):
  File "/home/zhangi/miniconda3/envs/perses-rbd-ace2/lib/python3.8/site-packages/mdtraj/core/trajectory.py", line 2003, in image_molecules
    _geometry.image_molecules(result.xyz, box, anchor_molecules_atom_indices, other_molecules_atom_indices, sorted_bonds)
ValueError: Buffer dtype mismatch, expected 'float' but got 'double'


ValueError: Buffer dtype mismatch, expected 'float' but got 'double'

...

Exception ignored in: 'mdtraj.geometry._geometry.image_frame'
Traceback (most recent call last):
  File "/home/zhangi/miniconda3/envs/perses-rbd-ace2/lib/python3.8/site-packages/mdtraj/core/trajectory.py", line 2003, in image_molecules
    _geometry.image_molecules(result.xyz, box, anchor_molecules_atom_indices, other_molecules_atom_indices, sorted_bonds)
ValueError: Buffer dtype mismatch, expected 'float' but got 'double'


ValueError: Buffer dtype mismatch, expected 'float' but got 'double'

Exception ignored in: 'mdtraj.geometry._geometry.image_frame'
Traceback (most recent call last):
  File "/home/zhangi/miniconda3/envs/perses-rbd-ace2/lib/python3.8/site-packages/mdtraj/core/trajectory.py", line 2003, in image_molecules
    _geometry.image_molecules(result.xyz, box, anchor_molecules_atom_indices, other_molecules_atom_indices, sorted_bonds)
ValueError: Buffer dtype mismatch, expected 'float' but got 'double'


ValueError: Buffer dtype mismatch, expected 'float' but got 'double'

Exception ignored in: 'mdtraj.geometry._geometry.image_frame'
Traceback (most recent call last):
  File "/home/zhangi/miniconda3/envs/perses-rbd-ace2/lib/python3.8/site-packages/mdtraj/core/trajectory.py", line 2003, in image_molecules
    _geometry.image_molecules(result.xyz, box, anchor_molecules_atom_indices, other_molecules_atom_indices, sorted_bonds)
ValueError: Buffer dtype mismatch, expected 'float' but got 'double'


In [17]:
traj

<mdtraj.Trajectory with 948 frames, 183585 atoms, 57059 residues, without unitcells at 0x2b76a60a02e0>

## 31/30/0 (with 34 replicas)

In [4]:
outdir = "/data/chodera/zhangi/perses_benchmark/repex/31/30/0/"
i = 0
phase = 'complex'
name = 'asn'

with open(os.path.join(outdir, f"{i}_{phase}.pickle"), "rb") as f:
    htf = pickle.load(f)
    


In [6]:
all_pos_new = make_traj(outdir, htf, i, phase, replica=24)


<ipython-input-3-1d6d24e74a6c>:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for iteration in tqdm_notebook(range(n_iter)):


  0%|          | 0/840 [00:00<?, ?it/s]

[24]
[27]
[23]
[27]
[25]
[27]
[27]
[23]
[25]
[25]
[23]
[26]
[26]
[28]
[27]
[24]
[25]
[26]
[23]
[28]
[24]
[24]
[24]
[29]
[27]
[24]
[24]
[25]
[25]
[27]
[25]
[28]
[25]
[25]
[27]
[27]
[31]
[27]
[25]
[28]
[27]
[27]
[26]
[25]
[25]
[23]
[23]
[24]
[23]
[29]
[24]
[25]
[24]
[24]
[24]
[26]
[27]
[27]
[31]
[25]
[23]
[26]
[26]
[29]
[23]
[23]
[23]
[26]
[23]
[25]
[26]
[25]
[25]
[25]
[29]
[26]
[25]
[26]
[25]
[27]
[25]
[25]
[24]
[25]
[25]
[25]
[28]
[25]
[28]
[28]
[23]
[28]
[24]
[25]
[26]
[29]
[23]
[26]
[26]
[26]
[25]
[25]
[25]
[25]
[26]
[24]
[24]
[25]
[29]
[25]
[25]
[23]
[25]
[23]
[24]
[29]
[23]
[25]
[28]
[25]
[26]
[25]
[25]
[25]
[25]
[25]
[23]
[23]
[29]
[27]
[27]
[27]
[27]
[28]
[29]
[26]
[23]
[23]
[25]
[28]
[23]
[26]
[28]
[27]
[23]
[27]
[28]
[25]
[26]
[25]
[25]
[25]
[28]
[25]
[25]
[25]
[29]
[23]
[24]
[25]
[25]
[26]
[29]
[30]
[30]
[30]
[25]
[24]
[24]
[23]
[23]
[25]
[25]
[26]
[25]
[25]
[24]
[25]
[25]
[24]
[24]
[25]
[25]
[25]
[25]
[24]
[28]
[27]
[27]
[23]
[23]
[23]
[27]
[26]
[24]
[24]
[24]
[25]
[23]
[23]


In [8]:
traj

<mdtraj.Trajectory with 906 frames, 183585 atoms, 57059 residues, without unitcells at 0x2b4d75f48220>

In [9]:
new_top = md.Topology.from_openmm(htf._topology_proposal.new_topology)

traj = md.Trajectory(np.array(all_pos_new), new_top)

box_vectors = np.array([val.value_in_unit_system(unit.md_unit_system) for val in htf.hybrid_system.getDefaultPeriodicBoxVectors()])

vectors = np.array([box_vectors]) 

# Set unit cell vectors in traj 
traj.unitcell_vectors = np.repeat(vectors, 906, axis=0)
traj.image_molecules(inplace=True)

traj.save(os.path.join(outdir, f"{i}_{phase}_new_replica_24.dcd"))
traj[0].save(os.path.join(outdir, f"{i}_{phase}_new_replica_24.pdb"))

ValueError: Buffer dtype mismatch, expected 'float' but got 'double'

Exception ignored in: 'mdtraj.geometry._geometry.image_frame'
Traceback (most recent call last):
  File "/home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/mdtraj/core/trajectory.py", line 2003, in image_molecules
    _geometry.image_molecules(result.xyz, box, anchor_molecules_atom_indices, other_molecules_atom_indices, sorted_bonds)
ValueError: Buffer dtype mismatch, expected 'float' but got 'double'


ValueError: Buffer dtype mismatch, expected 'float' but got 'double'

Exception ignored in: 'mdtraj.geometry._geometry.image_frame'
Traceback (most recent call last):
  File "/home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/mdtraj/core/trajectory.py", line 2003, in image_molecules
    _geometry.image_molecules(result.xyz, box, anchor_molecules_atom_indices, other_molecules_atom_indices, sorted_bonds)
ValueError: Buffer dtype mismatch, expected 'float' but got 'double'


ValueError: Buffer dtype mismatch, expected 'float' but got 'double'

...

Exception ignored in: 'mdtraj.geometry._geometry.image_frame'
Traceback (most recent call last):
  File "/home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/mdtraj/core/trajectory.py", line 2003, in image_molecules
    _geometry.image_molecules(result.xyz, box, anchor_molecules_atom_indices, other_molecules_atom_indices, sorted_bonds)
ValueError: Buffer dtype mismatch, expected 'float' but got 'double'


ValueError: Buffer dtype mismatch, expected 'float' but got 'double'

Exception ignored in: 'mdtraj.geometry._geometry.image_frame'
Traceback (most recent call last):
  File "/home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/mdtraj/core/trajectory.py", line 2003, in image_molecules
    _geometry.image_molecules(result.xyz, box, anchor_molecules_atom_indices, other_molecules_atom_indices, sorted_bonds)
ValueError: Buffer dtype mismatch, expected 'float' but got 'double'


ValueError: Buffer dtype mismatch, expected 'float' but got 'double'

Exception ignored in: 'mdtraj.geometry._geometry.image_frame'
Traceback (most recent call last):
  File "/home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/mdtraj/core/trajectory.py", line 2003, in image_molecules
    _geometry.image_molecules(result.xyz, box, anchor_molecules_atom_indices, other_molecules_atom_indices, sorted_bonds)
ValueError: Buffer dtype mismatch, expected 'float' but got 'double'
